In [20]:
# Stats on Ca2+ imaging with miniscope and Osc
#######################################################################################
                            # Define Experiment type #
#######################################################################################

DrugExperiment=0 # 1 if CGP Experiment, 0 if Baseline Experiment

suffix='_AH_DS'

Local=1

#choosed_folder='Osc_2024_08_04_10_35_33_AH_FINAL' if DrugExperiment else 'Osc_2024_08_04_09_24_52_AH_FINAL'
choosed_folder1='Osc_2024_09_04_14_58_48_AH_DS' # for Baseline Expe
choosed_folder2='Osc_2024_09_04_17_05_19_AH_DS' # for CGP Expe

PrefVigExcel_file = 'AVG_VigSt_2024-09-03_11_16_02_AB_wRealTS_forCGP' if DrugExperiment else 'AVG_VigSt_2024-09-03_11_00_49_AB_wRealTS'
#PrefVigExcel_file = 'AVG_VigSt_2024-08-22_13_50_33_AB_Clustering_forCGP' if DrugExperiment else 'AVG_VigSt_2024-08-22_13_28_19_AB_Clustering'

#######################################################################################
                                # Load packages #
#######################################################################################

import statsmodels.api as sm
import quantities as pq
import numpy as np
import math 
import neo
import json
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import pickle
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import zscore
from scipy.stats import sem
from datetime import datetime
import shutil
from scipy import stats
from scipy.stats import t
from scipy.stats import norm

import warnings
warnings.filterwarnings("ignore")


def column_with_max_single_per_row(row):
    max_val = row.max()  # Find the max value in the row
    max_columns = row == max_val  # Boolean Series of columns with max value
    
    if max_columns.sum() > 1:
        return 'NoPref'  # More than one column has the max value
    else:
        return max_columns.idxmax()  # Return the name of the column with max value

def divide_keys(data):
    it=list(data.keys())[0]
    d=data[it]
    data[it]=d.replace(0, np.nan)
    for sheet in list(data.keys())[1:]: 
        data[sheet].div(data[it][sheet], axis=0)
    del data[it]
    return data    

########################################################################
        # SCRIPT 27AB_GrandAverages&Stats_for_Osc
########################################################################

# Specify the directory containing the Excel files
InitialDirectory1 = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/AB_Analysis" if Local else "/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/AB_Analysis" 
directory1= f'{InitialDirectory1}/{choosed_folder1}'
InitialDirectory2 ="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/CGP/AB_Analysis" if Local else "/crnldata/waking///L1imaging/AnalysedMarch2023/Gaelle/CGP/AB_Analysis"
directory2= f'{InitialDirectory2}/{choosed_folder2}'

# Get the current date and time
FolderNameSave=str(datetime.now())[:19]
FolderNameSave = FolderNameSave.replace(" ", "_").replace(".", "_").replace(":", "_")
destination_folder= f"//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/AB_GlobalAnalysis/AVG_Osc_{FolderNameSave}{suffix}/"
os.makedirs(destination_folder)
folder_to_save=Path(destination_folder)

# Copy the script file to the destination folder
source_script = "C:/Users/Manip2/SCRIPTS/CodePythonAudrey/CodePythonAurelie/HayLabAnalysis/python/25_28_GrandAverage&Stats_for_DownStatesSpdl&SWR_FullAuto.ipynb"
destination_file_path = f"{destination_folder}/25_28_GrandAverage&Stats_for_DownStatesSpdl&SWR_FullAuto.txt"
shutil.copy(source_script, destination_file_path)

NrSubtypeList=['L2&3', 'L1']
CTX=['PFC_', 'S1_', 'S1PFC_']
CTX2=['PFC', 'S1', 'S1PFC']
Coupling=['', 'UnCoupled', 'Coupled']
OscList=['Spdl', 'SWR', 'DS']
Drugs= ['Baseline', 'CGP'] if DrugExperiment else ['Baseline']

for o, Osc in enumerate(OscList): 
    
    print(Osc, 'oscillations analysis...')

    AllOscStatut=pd.DataFrame()
    AllOscDuration=pd.DataFrame()
    AllOscCoupling=pd.DataFrame()
    AllGlobalSpindle=pd.DataFrame()
    AllOscStartLocation=pd.DataFrame()
    
    AllOscStatutS1=pd.DataFrame()
    AllOscDurationS1=pd.DataFrame()
    AllOscCouplingS1=pd.DataFrame()

    AllOscStatutPFC=pd.DataFrame()
    AllOscDurationPFC=pd.DataFrame()
    AllOscCouplingPFC=pd.DataFrame()

    AllOscStatutS1PFC=pd.DataFrame()
    AllOscDurationS1PFC=pd.DataFrame()
    AllOscCouplingS1PFC=pd.DataFrame()

    for NrSubtype in NrSubtypeList: 
        
        print('... for', NrSubtype, 'neurons...')
        
        # Initialize an empty list to store the dataframes
        dfs = []
        filtered_df=[]

        if NrSubtype=='L1':
            MiceList=['BlackLinesOK', 'BlueLinesOK', 'GreenDotsOK', 'GreenLinesOK', 'RedLinesOK']
        else:
            MiceList=['Purple', 'ThreeColDotsOK', 'ThreeBlueCrossesOK']

        nametofind=f'{Osc}_Global'

        ###########################################################################
                                    ##### GLOBAL #####
        ###########################################################################

        # Recursively traverse the directory structure
        for directory in [directory1, directory2]: #both Baseline & CGP experiments 
            for root, _, files in os.walk(directory):
                for filename in files:
                    # Check if the file is an Excel file and contains the specified name
                    if filename.endswith('.pkl') and nametofind in filename:
                        if any(name in filename for name in MiceList):  
                            # Construct the full path to the file
                            filepath = os.path.join(root, filename)
                            # Read the file and append it to the list
                            with open(filepath, 'rb') as pickle_file:
                                df = pickle.load(pickle_file)
                            dfs.append(df)
                            print(filename)

        # Concatenate all dataframes into a single dataframe
        combined_df = pd.concat(dfs, ignore_index=True)

        combined_df['Unique_Unit'] = combined_df['Unique_Unit'].astype(str)
        combined_df['UnitNumber'] = combined_df['UnitNumber'].astype(str)
        combined_df['UnitValue'] = combined_df['UnitValue'].astype(str)

        combined_df[f'{Osc}Statut'] = combined_df[f'{Osc}Statut'].astype(str)

        combined_df['Unit_ID'] = combined_df['Mice'] + combined_df['Unique_Unit']
        combined_df['Session_ID'] = combined_df['Mice'] + combined_df['Session'].astype(str)

        unique_count = combined_df['Unit_ID'].nunique()
        print(unique_count, f'{NrSubtype} neurons recorded') 

        # Remove non defined Unique Units 
        combined_df = combined_df[combined_df['Unique_Unit'] != '[]']
        combined_df = combined_df.dropna(subset=['Unique_Unit'])
        unique_count = combined_df['Unit_ID'].nunique()
        print(unique_count, f'{NrSubtype} neurons in the cross-registration') 
        
        combined_df[f'{Osc}_ID'] = combined_df['Mice'] + combined_df['Session'] + combined_df[f'{Osc}Number'].astype(str)
        
        unique_count = combined_df[f'{Osc}_ID'].nunique()
        print(unique_count, f'{Osc} recorded in total')

        filenameOut = f'{folder_to_save}/{NrSubtype}_{Osc}_Global.xlsx'
        writer = pd.ExcelWriter(filenameOut)
        combined_df.to_excel(writer)
        writer.close()


        ###########################################################################
                                    ##### PSTH #####
        ###########################################################################

        Data=['Ca', 'Sp']
        for data in Data:
            for drug in Drugs: 
                for ctx in CTX2:
                    for coup in Coupling:  

                        ctx= '' if Osc=='SWR' else ctx  
                        
                        locals()[f'{data}PSTH{Osc}_{coup}{ctx}{drug}']={}
                        dfsPSTH_per_sheet = locals()[f'{data}PSTH{Osc}_{coup}{ctx}{drug}']
                        nametofind2=f'{Osc}_{data}PSTH_{coup}{ctx}{drug}'
        
                        # Recursively traverse the directory structure
                        for directory in [directory1]:
                            for root, _, files in os.walk(directory):
                                for filename in files:
                                    if filename.endswith('.pkl') and nametofind2 in filename: 
                                        if any(name in filename for name in MiceList): 
                                            # Construct the full path to the file
                                            filepath = os.path.join(root, filename)
                                            with open(filepath, 'rb') as pickle_file:
                                                df = pickle.load(pickle_file)
                                            for key, value in df.items():
                                                if key in dfsPSTH_per_sheet:
                                                    dfsPSTH_per_sheet[key]=pd.concat([dfsPSTH_per_sheet[key],value], ignore_index=False, axis=0)
                                                else:
                                                    dfsPSTH_per_sheet[key]=value


        ###########################################################################
                                ##### PREFERENCE #####
        ########################################################################### 

        for drug in Drugs: 
            
            combined_df_Drug = combined_df.copy()
            try:
                combined_df_Drug = combined_df_Drug[combined_df_Drug['Drug'] == drug]
            except: 
                combined_df_Drug=combined_df_Drug
            
            folder_to_save2= f'{folder_to_save}/{drug}/'
            if NrSubtype=='L2&3' and o==0 :
                os.makedirs(folder_to_save2)
                        
            # Load the Excel file and read each sheet into a separate DataFrame
            excel_file = f'//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/AB_GlobalAnalysis/{PrefVigExcel_file}/Baseline/{NrSubtype}_ActivityPreference.xlsx'
            sheets = pd.read_excel(excel_file, sheet_name=None, header=None, index_col=0)  # sheet_name=None reads all sheets
            AllUnits=combined_df_Drug['Unit_ID'].unique()
            sheets['AllUnits']=pd.DataFrame(AllUnits)

            # Print the names of the sheets and their corresponding DataFrames
            for List_name, listI in sheets.items():
                
                thelist=listI[0].tolist() if List_name=='AllUnits' else listI[1].tolist()
                filtered_df = combined_df_Drug[combined_df_Drug['Unit_ID'].isin(thelist)]
                List_name=List_name.replace( '\\', '/')

                if NrSubtype=='L2&3' and o==0:
                    new_folder= f"{folder_to_save2}/{List_name}/"
                    os.makedirs(new_folder)

                ###########################################################################
                                        ##### AVERAGE PSTH #####
                ###########################################################################

                filenameOutAVG = f'{folder_to_save2}/{List_name}/{NrSubtype}_{Osc}_AVG&SEM.xlsx'
                excel_writerAVG = pd.ExcelWriter(filenameOutAVG)

                filenameOutMean = f'{folder_to_save2}/{List_name}/{NrSubtype}_{Osc}_During.xlsx'
                excel_writerMean= pd.ExcelWriter(filenameOutMean)
                
                for data in Data:

                    BigArray=pd.DataFrame()
                    BigArrayBSL=pd.DataFrame()
                    AVGArray=pd.DataFrame()
                    AVGArrayBSL=pd.DataFrame()

                    filenameOut = f'{folder_to_save2}/{List_name}/{NrSubtype}_{Osc}_{data}PSTH.xlsx'
                    excel_writer = pd.ExcelWriter(filenameOut)

                    for ctx in CTX2:
                        for coup in Coupling:    
                            
                            ctx= '' if Osc=='SWR' else ctx
                            df_per_sheet=locals()[f'{data}PSTH{Osc}_{coup}{ctx}{drug}']

                            activatedNr=0
                            inhibitedNr=0
                            modulatedNr=0

                            if len(df_per_sheet.keys()): # not empty

                                arrayLen = 0
                                for key, value in df_per_sheet.items():
                                    if value.size > 0:
                                        arrayLen = np.shape(value)[1]
                                        break  

                                Array=pd.DataFrame(columns=np.arange(0,arrayLen,1))

                                for nr in df_per_sheet.keys():
                                    
                                    if df_per_sheet[nr].shape[0] > 10 : # at least 10 oscillations recorded

                                        AVGtrace=np.nanmean((df_per_sheet[nr]), axis=0)
                                        BaselineSTD=np.std(AVGtrace[:df_per_sheet[nr].shape[1]//4], axis=0)

                                        upperThrs=np.nanmean(AVGtrace[:df_per_sheet[nr].shape[1]//4], axis=0) + BaselineSTD*2
                                        lowerThrs=np.nanmean(AVGtrace[:df_per_sheet[nr].shape[1]//4], axis=0) - BaselineSTD*2

                                        AVGbefore=np.nanmean(AVGtrace[df_per_sheet[nr].shape[1]//4:df_per_sheet[nr].shape[1]//4*2], axis=0)
                                        AVGearly=np.nanmean(AVGtrace[df_per_sheet[nr].shape[1]//4*2:df_per_sheet[nr].shape[1]//4*3], axis=0)
                                        AVGlate=np.nanmean(AVGtrace[df_per_sheet[nr].shape[1]//4*3:df_per_sheet[nr].shape[1]//4*4], axis=0)

                                        if AVGbefore > upperThrs or AVGearly > upperThrs or AVGlate > upperThrs :
                                            activatedNr +=1
                                            modulatedNr=1
                                        if AVGbefore < lowerThrs or AVGearly < lowerThrs or AVGlate < lowerThrs :
                                            inhibitedNr +=1
                                            modulatedNr=1

                                        Array.loc[nr]=AVGtrace
                                
                                # Only keep the neurons belonging to the list (All units, NREM active, REM active, etc)
                                present_indices = [idx for idx in thelist if idx in Array.index]
                                Array = Array.loc[present_indices] 

                                # Leave a blanck space for units not recorded in that condition
                                missing_indexes = set(thelist) - set(Array.index)
                                Array = Array.reindex(Array.index.union(missing_indexes))
                                Array = Array.sort_index()

                                # Reduce the array by half
                                #Array=Array.iloc[:, Array.shape[1] // 4 : Array.shape[1] // 4 * 3] # -2.5 to 2.5 sec
                                Array=Array.iloc[:, Array.shape[1] // 10 *4  : Array.shape[1] // 10 * 6] # -1 to 1 sec

                                Array.to_excel(excel_writer, sheet_name=f'{ctx}{coup}{Osc}', index=True, header=False)

                                mArray=Array.mean(axis=0)
                                semArray = stats.sem(Array, axis=0, ddof=1, nan_policy='omit')
                                icArray = norm.ppf((1 +  0.95) / 2) * (np.std(Array, axis=0) / np.sqrt(Array.shape[0]))
                                SmallArray=pd.DataFrame(np.transpose([mArray,semArray,icArray]), columns=[f'{ctx}{coup}{Osc} Mean', f'{ctx}{coup}{Osc} SEM', f'{ctx}{coup}{Osc} IC'])
                                BigArray=pd.concat([BigArray,SmallArray], axis=1)

                                SecondHalf_columns = Array.iloc[:, Array.shape[1] //2:]
                                mean_baseline = SecondHalf_columns.mean(axis=1)
                                avgarray=pd.DataFrame(mean_baseline, columns=[f'{ctx}{coup}{Osc}'])
                                AVGArray=pd.concat([AVGArray,avgarray], axis=1)
                                
                                # Baseline signals
                                baseline_columns = Array.iloc[:, :Array.shape[1] // 4]
                                mean_baseline = baseline_columns.mean(axis=1)
                                Array = Array.sub(mean_baseline, axis=0)
                                Array.to_excel(excel_writer, sheet_name=f'BSL_{ctx}{coup}{Osc}', index=True, header=False)

                                mArray=Array.mean(axis=0)
                                semArray = stats.sem(Array, axis=0, ddof=1, nan_policy='omit')
                                icArray = norm.ppf((1 +  0.95) / 2) * (np.std(Array, axis=0) / np.sqrt(Array.shape[0]))
                                SmallArray=pd.DataFrame(np.transpose([mArray,semArray,icArray]), columns=[f'{ctx}{coup}{Osc} Mean', f'{ctx}{coup}{Osc} SEM', f'{ctx}{coup}{Osc} IC'])
                                BigArrayBSL=pd.concat([BigArrayBSL,SmallArray], axis=1)

                                SecondHalf_columns = Array.iloc[:, Array.shape[1]//2:]
                                mean_baseline = SecondHalf_columns.mean(axis=1)
                                avgarray=pd.DataFrame(mean_baseline, columns=[f'{ctx}{coup}{Osc}'])
                                AVGArrayBSL=pd.concat([AVGArrayBSL,avgarray], axis=1)

                    excel_writer.close()

                    BigArrayBSL.to_excel(excel_writerAVG, sheet_name=f'BSL_{data}PSTH', index=True, header=True)
                    AVGArrayBSL.to_excel(excel_writerMean, sheet_name=f'BSL_{data}PSTH', index=True, header=True)

                    BigArray.to_excel(excel_writerAVG, sheet_name=f'{data}PSTH', index=True, header=True)
                    AVGArray.to_excel(excel_writerMean, sheet_name=f'{data}PSTH', index=True, header=True)

                excel_writerAVG.close()                   
                excel_writerMean.close()                   

        #######################
        # Propreties Osc
        #######################
        filenameOut = f'{folder_to_save}/{Osc}Propreties.xlsx'
        writer = pd.ExcelWriter(filenameOut)

        combined_dfOsc = combined_df.drop_duplicates(subset=f'{Osc}_ID', keep='first')
        #All Spdl
        OscStatut = pd.crosstab(index=combined_dfOsc['Session_ID'],columns=[combined_dfOsc['Drug'], combined_dfOsc[f'{Osc}Statut']])
        AllOscStatut=pd.concat([AllOscStatut, OscStatut], axis=0)
        OscDuration = combined_dfOsc.pivot_table(index='Session_ID', columns='Drug', values=f'{Osc}Duration', aggfunc='mean')
        AllOscDuration=pd.concat([AllOscDuration, OscDuration], axis=0)
        if Osc== 'Spdl' or Osc== 'SWR':
            OscCoupling = pd.crosstab(index=combined_dfOsc['Session_ID'],columns=[combined_dfOsc['Drug'], combined_dfOsc[f'SWR_inside_Spdl']])
            AllOscCoupling=pd.concat([AllOscCoupling, OscCoupling], axis=0)

        if Osc== 'Spdl':
            GlobalSpindle = pd.crosstab(index=combined_dfOsc['Session_ID'],columns=[combined_dfOsc['Drug'], combined_dfOsc[f'GlobalSpindle']])
            AllGlobalSpindle=pd.concat([AllGlobalSpindle, GlobalSpindle], axis=0)
            OscStartLocation = pd.crosstab(index=combined_dfOsc['Session_ID'],columns=[combined_dfOsc['Drug'], combined_dfOsc[f'{Osc}StartLocation']])
            AllOscStartLocation=pd.concat([AllOscStartLocation, OscStartLocation], axis=0)
            """
            #S1 Spdl
            combined_dfOscS1=combined_dfOsc[combined_dfOsc[f'{Osc}StartLocation']=='S1' and combined_dfOscS1[f'GlobalSpindle']=='Local']
            OscStatut = pd.crosstab(index=combined_dfOscS1['Session_ID'],columns=[combined_dfOscS1['Drug'], combined_dfOscS1[f'{Osc}Statut']])
            AllOscStatutS1=pd.concat([AllOscStatut, OscStatut], axis=0)
            OscDuration = combined_dfOscS1.pivot_table(index='Session_ID', columns='Drug', values=f'{Osc}Duration', aggfunc='mean')
            AllOscDurationS1=pd.concat([AllOscDuration, OscDuration], axis=0)
            OscCoupling = pd.crosstab(index=combined_dfOscS1['Session_ID'],columns=[combined_dfOscS1['Drug'], combined_dfOscS1[f'SWR_inside_Spdl']])
            AllOscCouplingS1=pd.concat([AllOscCoupling, OscCoupling], axis=0)

            #PFC Spdl
            combined_dfOscPFC=combined_dfOsc[combined_dfOsc[f'{Osc}StartLocation']=='PFC' and combined_dfOscS1[f'GlobalSpindle']=='Local']
            OscStatut = pd.crosstab(index=combined_dfOscPFC['Session_ID'],columns=[combined_dfOscPFC['Drug'], combined_dfOscPFC[f'{Osc}Statut']])
            AllOscStatutPFC=pd.concat([AllOscStatut, OscStatut], axis=0)
            OscDuration = combined_dfOscPFC.pivot_table(index='Session_ID', columns='Drug', values=f'{Osc}Duration', aggfunc='mean')
            AllOscDurationPFC=pd.concat([AllOscDuration, OscDuration], axis=0)
            OscCoupling = pd.crosstab(index=combined_dfOscPFC['Session_ID'],columns=[combined_dfOscPFC['Drug'], combined_dfOscPFC[f'SWR_inside_Spdl']])
            AllOscCouplingPFC=pd.concat([AllOscCoupling, OscCoupling], axis=0)

            #S1PFC Spdl
            combined_dfOscS1PFC=combined_dfOsc[combined_dfOsc[combined_dfOscS1[f'GlobalSpindle']=='Global']]
            OscStatut = pd.crosstab(index=combined_dfOscS1PFC['Session_ID'],columns=[combined_dfOscS1PFC['Drug'], combined_dfOscS1PFC[f'{Osc}Statut']])
            AllOscStatutS1PFC=pd.concat([AllOscStatut, OscStatut], axis=0)
            OscDuration = combined_dfOscS1PFC.pivot_table(index='Session_ID', columns='Drug', values=f'{Osc}Duration', aggfunc='mean')
            AllOscDurationS1PFC=pd.concat([AllOscDuration, OscDuration], axis=0)
            OscCoupling = pd.crosstab(index=combined_dfOscS1PFC['Session_ID'],columns=[combined_dfOscS1PFC['Drug'], combined_dfOscS1PFC[f'SWR_inside_Spdl']])
            AllOscCouplingS1PFC=pd.concat([AllOscCoupling, OscCoupling], axis=0)
            """    

    AllOscStatut.to_excel(writer, sheet_name=f'CouplingStatut')
    AllOscDuration.to_excel(writer, sheet_name=f'MeanDuration')
    if Osc== 'Spdl' or Osc== 'SWR':
        AllOscCoupling.to_excel(writer, sheet_name=f'SWR_inside_Spdl')    
    if Osc== 'Spdl':
        AllGlobalSpindle.to_excel(writer, sheet_name=f'GlobalSpindle')
        AllOscStartLocation.to_excel(writer, sheet_name=f'StartLocation')
        """
        AllOscStatutS1.to_excel(writer, sheet_name=f'S1_CouplingStatut')
        AllOscDurationS1.to_excel(writer, sheet_name=f'S1_MeanDuration')
        AllOscCouplingS1.to_excel(writer, sheet_name=f'S1_SWR_inside_Spdl') 
        
        AllOscStatutPFC.to_excel(writer, sheet_name=f'PFC_CouplingStatut')
        AllOscDurationPFC.to_excel(writer, sheet_name=f'PFC_MeanDuration')
        AllOscCouplingPFC.to_excel(writer, sheet_name=f'PFC_SWR_inside_Spdl') 
        
        AllOscStatutS1PFC.to_excel(writer, sheet_name=f'S1PFC_CouplingStatut')
        AllOscDurationS1PFC.to_excel(writer, sheet_name=f'S1PFC_MeanDuration')
        AllOscCouplingS1PFC.to_excel(writer, sheet_name=f'S1PFC_SWR_inside_Spdl')
        """

    writer.close()

SWR oscillations analysis...
... for L2&3 neurons...
SWR_Global_Purple.pkl
SWR_Global_ThreeBlueCrossesOK.pkl
SWR_Global_ThreeColDotsOK.pkl
333 L2&3 neurons recorded
332 L2&3 neurons in the cross-registration
8554 SWR recorded in total
CaPSTHSWR_Baseline
CaPSTHSWR_UnCoupledBaseline
CaPSTHSWR_CoupledBaseline
CaPSTHSWR_Baseline
CaPSTHSWR_UnCoupledBaseline
CaPSTHSWR_CoupledBaseline
CaPSTHSWR_Baseline
CaPSTHSWR_UnCoupledBaseline
CaPSTHSWR_CoupledBaseline
SpPSTHSWR_Baseline
SpPSTHSWR_UnCoupledBaseline
SpPSTHSWR_CoupledBaseline
SpPSTHSWR_Baseline
SpPSTHSWR_UnCoupledBaseline
SpPSTHSWR_CoupledBaseline
SpPSTHSWR_Baseline
SpPSTHSWR_UnCoupledBaseline
SpPSTHSWR_CoupledBaseline
... for L1 neurons...
SWR_Global_BlackLinesOK.pkl
SWR_Global_BlueLinesOK.pkl
SWR_Global_GreenDotsOK.pkl
SWR_Global_GreenLinesOK.pkl
SWR_Global_RedLinesOK.pkl
SWR_Global_BlackLinesOK.pkl
129 L1 neurons recorded
129 L1 neurons in the cross-registration
6681 SWR recorded in total
CaPSTHSWR_Baseline
CaPSTHSWR_UnCoupledBaseline
Ca